## 与えられた手書き文字に対してガウスカーネルに対する最小二乗回帰により, パターン認識を行え

必要なもののインポート

In [2]:
import csv
import numpy as np
import time
from sklearn.metrics import classification_report

データは先端データ解析の授業のものを再利用

In [3]:
"""
必要なファイル読み込み
"""
datas = []
for i in range(10):
    train_file = "./digit_csv/digit_train" + str(i) + ".csv"
    data  = np.loadtxt(train_file, delimiter=",")
    datas.append(data)
print(datas[0].shape)
test_datas = []
for i in range(10):
    test_file = "./digit_csv/digit_test" + str(i) + ".csv"
    data  = np.loadtxt(test_file, delimiter=",")
    test_datas.append(data)
test_X = test_datas[:]
test_X =  np.r_[test_X[0], test_X[1], test_X[2], test_X[3], test_X[4], test_X[5], test_X[6], test_X[7], test_X[8], test_X[9] ]
print(test_X.shape)

print(len(data))
# data = data[:, :, np.newaxis] #i で埋める必要
t = np.matrix(np.ones(500)).T
f = np.matrix(np.ones(500*9)*(-1)).T
y = np.r_[t, f]


(500, 256)
(2000, 256)
200


学習と分類をする

### """制約付き最小二乗法"""
def rols(y, X, lmd = 0):
    Xsq = (X.T).dot(X)
    res = np.linalg.inv( Xsq + lmd*np.identity(Xsq.shape[0])).dot(X.T).dot(y)
    return res


"""ガウスカーネル 　第二引数が学習データ"""
def gaus(X, c, h):
    rownum = X.shape[0]
    rownum2 = c.shape[0]
    K = np.zeros([rownum, rownum2])
    for i in range(0, rownum):
            for j in range(0, rownum2):
                norm = np.power(np.linalg.norm(X[i] - c[j]), 2)
                k = np.exp(-norm/(2*np.power(h, 2)))
                K[i][j] = k
    return K



"""訓練"""
start = time.time()#実行時間測定開始

scores = []
#10クラス分の判別をループで行う
for i in range(10):
    h = 0.5    #損失の重み

    print(u"クラス", str(i), u"への分類")
    X = datas[:]
    X1 = X.pop(i)
    X2 = np.r_[X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8] ]
    #最初の５００行がiの、残り４５００行がそれ以外のクラス判定になるように並び替え
    X3 = np.r_[X1, X2]

    print("カーネル回帰でパラメータ推定")
    k = gaus(X3, X3, h)

    theta = rols(y, k)
    print("回帰式にテストデータ代入、スコア算出")
    K = gaus(test_X, X3, h)
    F = K.dot(theta)
    print(F.shape)
    print("スコア格納")
    scores.append(F)
    print("スコアの形状：", F.shape)
    #実行時間測定終了、表示
    elapsed_time = time.time() - start
    print(str(i), u"番目へのクラス判別: ", elapsed_time/60)

#実行時間測定終了、表示
elapsed_time = time.time() - start
print("処理にかかった時間: ", elapsed_time/60)

In [6]:
#算出したスコアを結合
score_mat =  np.c_[ scores[0], scores[1], scores[2], scores[3], \
                   scores[4], scores[5], scores[6], scores[7], scores[8], scores[9]] 

In [7]:
#推定結果をベクトルにする
pred = []
for i in score_mat:
    pred.append(np.argmax(i))

#正しい答えのベクトルを作成
s =np.array([ [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]).T
true_val = np.kron(s, np.matrix(np.ones(200)).T)

print("分類精度を算出")
print(classification_report(true_val, pred))#, target_names=target_names))

分類精度を算出
             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       200
        1.0       1.00      1.00      1.00       200
        2.0       0.96      0.96      0.96       200
        3.0       0.95      0.97      0.96       200
        4.0       0.97      0.95      0.96       200
        5.0       0.96      0.94      0.95       200
        6.0       0.99      0.97      0.98       200
        7.0       0.98      0.96      0.97       200
        8.0       0.95      0.94      0.94       200
        9.0       0.92      0.97      0.95       200

avg / total       0.97      0.97      0.97      2000



**#結果  
precison, recall 共に97％の精度で推定出来ている**